## 주차요금 문제

In [ ]:
# 문제 : 주차요금 계산
# ref) https://school.programmers.co.kr/learn/courses/30/lessons/92341?language=python3
# 특징 ) 여러 경우에 대해서 꼼꼼하게 처리하는 게 중요 (시간 고려)


In [ ]:
# ---> 코드 정리
def solution(fees, records):
    answer = []
    base_time, base_cost, add_time, add_cost = fees
    park_dict = {} # 입력 주차 정보 정리를 위한 dict
    for record in records: # [ "01:23 0000 IN", "02:33 0000 out"]
        park_time, car_id, status = record.split(" ")
        car_id = int(car_id)
        car_record = [ convert_h_to_m(park_time),status ]
        if car_id in park_dict:
            park_dict[car_id].append( car_record)
        else:
            park_dict[car_id] = [car_record]
    car_dict = {} # 키:차번, 값:요금
    for car, logs in park_dict.items(): # k, v
        out_time_list = [ m for m, sta in logs if sta=="OUT"]
        in_time_list  = [m for m, sta in logs if sta=="IN"]
        if len(logs) % 2 == 1: # 출차 정보가 없는 기록들...
            out_time_list.append( convert_h_to_m("23:59"))
        total_min = sum(out_time_list) - sum(in_time_list)
        if total_min <= base_time:
            car_dict[car] = base_cost
        else: # 추가 요금 발생!!!!!
            cost = 0
            cost += base_cost
            total_min -= base_time # 누적시간 = 누적시간 -기본시간
            over_time_base = -( total_min * (-1) // add_time)
            cost += over_time_base * add_cost
            car_dict[car] = cost
    car_cost = dict(sorted(car_dict.items())) # key 정렬
    answer = list(car_cost.values())
    return answer

In [ ]:
# 정리)
# ---> 기본적인 문법이 복잡한 것은 없는 문제
# ---> 여러 케이스별로 꼼꼼하게 구현, 논리 구조 코드화!!!!!!!
# ---> 내가 필요한 정보들에 대한 세팅, 자료형 설정,,,
# ---> ++ 주어진 조건에 맞도록 정렬하는 부분!!!

## 얼음담기 문제(DFS/BFS)

In [ ]:
# 1) 왜 이 문제가 탐색인가?
#     =? 카운팅에 대한 조건
#        0으로 시작할 수 있따면 0으로 연결된 거 다 찾아서 1로 바꿔w줘(LRUD)
#     -> 주어진 환경에 맞게 니가 다 찾아줘라.

# 2) 탐색이라고 하면 여러 탐색 알고리즘 중에서 DFS, BFS? 다른거? 선택!
#     => 모두 다 사용이 가능한 방법!
#         DFS, BFS 크게 상관 없음.
#     => BFS는 어제 했으니 직접 하고
#         DFS 재귀함수를 활용해서 해보기

# 3) 이 문제의 해결하는 큰 틀이 뭐지?
#     => 카운팅! 특정 조건이 존재할 때마다 카운팅
#         카운팅에 대한 조건  : 0으로 상하좌우로 연결된 덩어리들이 될 때마다
#         (0으로 이어진 덩어리가 몇 개인지 찾는 게)
#     -> 조건 : 그 지점에서 일단 얼음을 ㅇ러리 수 있다면
#               다 얼릴 수 있을 때까지 얼리고
#               다 얼리면 +1 카운팅

In [ ]:
# BFS로 구현을 할 때 앞의 지도 k도시, 미로 탈출 최단 거리 문제와 차이
# => 앞의 문제는 출발점이 딱 1개 주어졌음
#     그 출발점에서 주구장창 다 탐색을 하면 끝
#     탐색 알고리즘을 1번만 돌리면 됨
# but) 이 문제는 어떤 차이냐?
# => 모든 n,m의 셀에서 탐색이 가능하면, 그 지점을 출발점으로 다 탐색해보 =
#     출발점이 1개가 아니라 2차원의 모든 n, m으로 시도를 해봐야!
# => 만약에 (n,m)지점에서 출발이 가능하다면 탐색하고 끝나면
#     카운팅을 +1로 올리고, 이 결과를 출력!


In [ ]:
# 시도1) BFS로 한다면, 대략적으로 흐름을 체크 해보겠음
# 큰 틀 : 모든 좌표에서 시작한다고 세팅!
#     for i in 가로줄:
#         for j in 세로줄:
#             => 2D 에서 (i,j)의 위치 : 탐색의 시작점을 부여할 수 있음.
#             if (i,j)로 시작할 수 있고, 다 탐색 끝나면:
#                 카운팅+1

# 세부적으로 보면,,,, 표현상으로 사람 인덱스..(0,0)-->(1,1)로 설명

# [출발점]
(1,1) ---> 시작 가능한 출발점이지 Check!!!!( i,j)값이 0인 경우만 시작
           ===> 시작가능하다고 하면
                도장 찍기 (1,1)값을 1로 도장찍으면 됨!!! 한 일에
           ===> (1,1)에서 LRUD 4가지 방향 다 가봐!! 할 일에 추가!!
                L = [(1,1)L, (1,1)R, (1,1)U, (1,1)D]
        ==> 다음 할 일을 선택 : BFS --> 앞에서 뽑으면 됨..
            (1,1)L ---> (0,1) :  outbound --> pass
            L = [ (1,1)R, (1,1)U, (1,1)D]
        ==> 다음 할 일을 선택 : BFS
            (1,1)R --> (1,2) : inbound & 얼음 얼릴 수 있는지 0
                 ==> (1,2)에 값을 1로 도장찍기!!!
                      L = [  (1,1)U, (1,1)D, (1,2)L, (1,2)R, (1,2)U, (1,2)D]
        ==> 다음 할 일 선택
            (1,1)U --> 아웃바운드 : pass
            L = [ (1,1)D, (1,2)L, (1,2)R, (1,2)U, (1,2)D]
        ==> 다음 할 일 선택
            (1,1)D ==> (2,1) :inbound & 얼을 얼릴 수 있는지 0
                  ==> (2,1)에 값을 1로 도장찍기!!!!
                      할 일을 추가
                      L = [  (1,2)L, (1,2)R, (1,2)U, (1,2)D, (2,1)L,(2,1)R, (2,1)U,(2,1)D]
        ==> 다음 할 일 선택
            (1,2)L ==> (1,1) : inboudn & 이미 얼음으로 1 --> pass
            L = [ (1,2)R, (1,2)U, (1,2)D, (2,1)L,(2,1)R, (2,1)U,(2,1)D]
        ==> 다음 할 일 선택
            (1,2)R ==> (1,3) : in & 이미 칸 --> pass
            L = [(1,2)U, (1,2)D, (2,1)L,(2,1)R, (2,1)U,(2,1)D]
        ==> 다음 할 일 선택
            (1,2)U ==> (0,2) : outbound ===> pass
            L = [ (1,2)D, (2,1)L,(2,1)R, (2,1)U,(2,1)D]
        ==> 다음 할 일 선택
            (1,2)D ==> (2,2) : in & 얼음 얼릴 수 있는 0
                ==> (2,2) 도장 1로 찍고
                    할 일 추가
                    L = [ (2,1)L,(2,1)R, (2,1)U,(2,1)D, (2,2)L, (2,2)R, (2,2)U, (2,2)D]
        ==> 다음 할 일 선택
            (2,1)L : pass
            (2,1)R : pass
            (2,1)U : pass
            (2,1)D,: pass
            (2,2)L,:pass
            (2,2)R : in & 얼음 얼릴 수 있는 0
                 ==> (2,3) 도장 1찍고
                     할 일 추가
                     L = [  (2,2)U, (2,2)D,(2,3)L,(2,3)R,(2,3)U,(2,3)D]
        ==> 다음 할 일 선택!!!
            (2,2)U : pass
            (2,2)D : pass
            (2,3)L :pass
            (2,3)R : pass
            (2,3)U : pass
            (2,3)D : pass
            L = []
            탐색이 종료!!!!!

(1,2) ==>시작할 수 없음
(1,3) ==> 시작할 수 없음
(1,4) ==> 시작할 수 없음
(1,5) ==> 시작할 수 있음.
         ==> 시작할 수 있으니 도장찍고
             할 일을 추가
             L =[ (1,5)L, (1,5)R, (1,5)U, (1,5)D]
        ==> 하나씩 선택
           (1,5)L : pass
           (1,5)R :pass
           (1,5)U : pass
           (1,5)D : pass
            L =[]
            탐색이 종료가 되었음.. +1 카운팅!!!
(2,1)
(2,2)
..
(4,5)

In [ ]:
# 문제 조건 세팅
graph_45 =[
    [0,0,1,1,0],
    [0,0,0,1,1],
    [1,1,1,1,1],
    [0,0,0,0,0]
]
n = 4
m = 5
graph_45

In [ ]:
# bfs로 하려고 세팅 : deque
from collections import deque

In [ ]:
# bfs로 탐색하는 함수
# 입력 : 출발점에 대한 정보(row, col) 좌표
# 기능 : 출발점에서 이동이 가능하다면,,,,,모두 이동하고, 체크하겠다!!
#        BFS 방식으로 다 체크하겠다!!!!!
# 출력 : 출발점에서 시작할 수 있고, 그래서 다 방문/탐색이 끝나면 1
#        출발점에서 시작할 수 없다면,,                           0
def ice_bfs(row_init, col_init):
    # 1) 주어진 시작점 좌표가 시작할 수 있는지 없는지 체크!!!!!
    if graph[row_init][col_init]==0: # 시작이 가능한!!!!
        # 2) 방문할 곳들에 대한 : 할 일에 대한 초기 세팅!!!!!!
        q = deque([row_init, col_init])
        #graph[row_init][col_init] = 1
        # --> 위에 초기 시작 점에 대한 도장을 할지 말지 체크!!!
        #     오해없이 하기 위해서는 찍고 가는 것을 추천!!!!!!

        # 참고) 한 일 graph에 직접 0을 1로 바꿔가면서 진행을 하려고 함!!!
        while True: # ---> 무한 루프........빠져나갈 세팅!!!!!
            # 종료 조건!!!! : q가 빈 값이 될 때 까지..
            if not q:
                return 1 # 시작은 가능하고, 할 것 다하고,,최종 결과 1
            # ---> 할 일에 대한 본격 세팅
            # 갈 곳에 대한 선택
            row, col = q.popleft() # bfs --> 왼쪽에서 선택, dfs--> q.pop()
            print(q)
            # 직접 LRUD에 대해서 이동에 대한 체크!!!!!!!!
            for dx, dy in [ [0,-1], [0,1], [1,0], [-1,0]]:
                # ---> 이동 가능한지 체크!!!! in/ out
                if 0 <= row+dx < n and 0 <= col+dy < m: # 지도 안
                    # ---> 얼릴 수 있는지 체크 : 1/0
                    if graph[row+dx][col+dy] == 0:
                        graph[row+dx][col+dy] = 1
                        q.append( [row+dx, col+dy ])

    else:
        # 주어진 좌표에서 시작조차 할 수 없는 시작점
        return 0

In [ ]:
# 주어진 문제를 BFS로 탐색을 할 때
graph = graph_45
result = 0 # 출력을 하는 카운팅 변수!!!!!
for row in range(n):
    for col in range(m):
        # 시작점 후보 (n,m)
        # if 시작할 수 있고, 탐색이 끝나면,,
        #  +1 카운팅 갱신!!!!!
        # ice_bfs(row, col)
        result += ice_bfs(row, col)
print(result)

In [ ]:
# ---> DFS 방식으로도 구현해도 별 문제는 없는 유형!!!!!
#      BFS에서 q.pop() 코드 변경을 해도.....
# ---> 재귀함수로 구현을 할 수 있다!!!!!
# 재귀함수 : 기능 중심으로 코드를 작성!!!! --> 점화식!!!!

In [ ]:
def dfs_recursive( row, col):
    # 기능 중심
    # 기능1) 주어진 점이 인바운드, 아웃바운드 체크
    if row <=-1 or row >=n or col<=-1 or col >=m: # out
        return False
    # 기능2) 인바운드이면,,,,얼음을 얼릴 수 있는지 체크!!!!!
    if graph[row][col] ==0:
        # 2-1) 도장찍기
        graph[row][col] = 1
        # 2-2) 더 갈 곳이 어딘지 할 일 추가: 할 일은 동일한데, 위치만 바꿔서.
        # 이 부분에 대해서 탐색의 방향성이나 우선순위가 있다면,,
        # 이 부분에 대해서 수정을 할 수 있음!!!
        # 아래 코드 : L 먼저 탐색하고, 혹시 R -> U ---> D
        dfs_recursive(row, col-1) # L : 그냥 제가 L을 먼저 했기에,,막히기 전까지 왼쪽으로 쭉~~~
        dfs_recursive(row, col+1) # R
        dfs_recursive(row-1, col) # U
        dfs_recursive(row+1, col) # D
        return True

    return False

In [ ]:
# DFS의 탐색이 가능하다면,,,,,재귀함수로도 구현할수 있지 않을까???
# ===> 기능 중심으로 좀 심플하게 구현이 가능하다!!!!!!

In [ ]:
# 문제 조건 세팅
graph_45 =[
    [0,0,1,1,0],
    [0,0,0,1,1],
    [1,1,1,1,1],
    [0,0,0,0,0]
]
n = 4
m = 5
graph_45

In [ ]:
graph = graph_45
result = 0
for row in range(n):
    for col in range(m):
        if dfs_recursive(row,col) == True:
            result += 1
print(result)

In [ ]:
# 정리
# DFS / BFS 모두 가능한 문제!!!!!!
# ===> 본인이 편한대로 선택해서 하시면 됨!!!!!!


# DFS 사용이 가능하다!!! : 스택방식( 재귀함수)
# ===> 1) 정의대로 꾸역꾸역 탐색 기본으로 구현을 하는 방식!!!
# ===> 2) 기능 중심으로 재귀함수로 구현도 가능함!!!!!


# BFS 상용이 가능하다!!! : 큐방식
# ==> 거리, 순서 중요할 수 있을 때
#     출발점에서 뭔가 점진적으로 진행할 때!!!!!
# ==> 최단거리 문제로 일반화 : 다익스트라 알고리즘
#     간단한 경우는 BFS로도 최단 거리가 해결이 됨!!!!
# ==> deque 속도적인 부분에서 체크!!!!
# 주의!!!1) 절대로 재귀함수로 구현하면 안 됨!!!!!!

In [ ]:
# 참고 : 여러 유형 중 dfs, bfs중 하나 예
# https://school.programmers.co.kr/learn/courses/30/lessons/92343
# 경우의 수 틱하게 dfs로 하는
# https://school.programmers.co.kr/learn/courses/30/lessons/92342
# ===> DFS/BFS 잘 쓰면 중간 killer들을 잘 넘어갈 수 있음!!!